In [2]:
import numpy,pandas
#For the preprocessing we need the data from Dream9.xlsx
Dream9_training=pandas.read_excel('/home/carlos/Documents/Intro2CompBio/Dream9.xlsx',"trainingData") 
Dream9_scoring=pandas.read_excel('/home/carlos/Documents/Intro2CompBio/Dream9.xlsx',"scoringData")
Dream9=pandas.concat([Dream9_training,Dream9_scoring])

#Division of types of Variables
All=list(Dream9_training.keys())
Sc=list(Dream9_scoring.keys())

#Dependent variables are present in the training set but not in the scoring set
Dependent=[]
for v in All:
    if v not in Sc:
        Dependent+=[v]

#Categorical variables have discrete values and can't be measured by euclidean distances
Categorical=['SEX', 'PRIOR.MAL', 'PRIOR.CHEMO', 'PRIOR.XRT', 'Infection', 'cyto.cat', 
             'ITD', 'D835', 'Ras.Stat', 'resp.simple', 'Relapse', 'vital.status']

#The last 231 variables are proteins
Protein=All[-231:]

Dream9_training[Categorical]

,SEX,PRIOR.MAL,PRIOR.CHEMO,PRIOR.XRT,Infection,cyto.cat,ITD,D835,Ras.Stat,resp.simple,Relapse,vital.status
train_id_001,F,YES,NO,YES,No,diploid,NEG,NEG,NEG,CR,No,A
train_id_002,F,NO,NO,NO,Yes,diploid,NEG,NEG,NEG,CR,Yes,D
train_id_003,F,YES,YES,YES,No,-5,NEG,NEG,NEG,RESISTANT,NaN,D
train_id_004,M,NO,NO,NO,Yes,IM,ND,ND,NotDone,CR,Yes,D
train_id_005,M,YES,NO,NO,No,Misc,NEG,NEG,NEG,CR,Yes,A
train_id_006,F,NO,NO,NO,No,diploid,NEG,NEG,NEG,CR,Yes,D
train_id_007,F,NO,NO,NO,Yes,diploid,NEG,NEG,NotDone,RESISTANT,NaN,A
train_id_008,F,NO,NO,NO,Yes,diploid,NEG,NEG,NotDone,CR,Yes,A
train_id_010,M,NO,NO,NO,No,Misc,POS,NEG,NEG,RESISTANT,NaN,A
train_id_011,F,NO,NO,NO,Yes,diploid,POS,NEG,NEG,CR,Yes,D


In [9]:
import numpy as np

def add_branch(Table,Dependent,Independent,Subset={},max_depth=-1):
    print '\nAnalyzing:',Dependent,Independent,Subset
    #Select only the subset
    Table=Table.copy()
    for key in Subset.keys():
        Table=Table[Table[key]==Subset[key]]
    #If the subset has only one value, return the value
    if len(Table[Dependent].unique())==1:
        print 'Determinated Leaf:', Table[Dependent].unique()[0]
        return Table[Dependent].unique()[0]
    #If there are no more independent variables, return the most common value
    if len(Independent)==0 or max_depth==0:
        print 'Indeterminated Leaf:\n', Table[Dependent].value_counts()
        return Table[Dependent].value_counts().idxmax()
    
    #Calculate the entropy of the subset
    freq=[]
    for dval in Table[Dependent].unique():
        freq+=[sum(Table[Dependent]==dval)]
    Freq=[float(f)/sum(freq) for f in freq]
    E=0
    for f in Freq:
        E+=-f*np.log(f)/np.log(2)
    print 'Subset Entropy:', E
    Vars=[]
    for ind in Independent:
        IG=E
        for ival in Table[ind].unique():
            SubTable=Table[Table[ind]==ival]
            #print SubTable
            freq=[]
            for dval in Table[Dependent].unique():
                freq+=[sum(SubTable[Dependent]==dval)]
            Freq=[float(f)/sum(freq) for f in freq]
            #print Freq
            ES=0
            for f in Freq:
                ES+=-f*np.log(f)/np.log(2) if f<>0 else 0
            #print ES
            IG-=float(len(SubTable))/len(Table)*ES
        print 'Information gain from %s: %f'%(ind,IG)
        Vars+=[(IG,ind)]
    #Find best branch
    Vars.sort(reverse=True)
    #Use this branch only if there if some information gain:
    if Vars[0][0]>0:
        Selected_branch=Vars[0][1]
    else:
        print 'Indeterminated Leaf:\n', Table[Dependent].value_counts()
        return Table[Dependent].value_counts().idxmax()
    print 'Branch:', Selected_branch
    #Make a new branch
    Independent.remove(Selected_branch)
    choices={}
    Subset=Subset.copy()
    for ival in Table[Selected_branch].unique():
            Subset.update({Selected_branch:ival})
            choices.update({ival:add_branch(Table,Dependent,Independent,Subset,max_depth-1)})
    return(Selected_branch,choices)

#Independent=list(Cars.keys())
#Independent.remove('Fast')
#Independent.remove('Model')
Independent=[v for v in Categorical if v not in Dependent]

add_branch(Dream9_training,'resp.simple',Independent,max_depth=5)


Analyzing: resp.simple ['SEX', 'PRIOR.MAL', 'PRIOR.CHEMO', 'PRIOR.XRT', 'Infection', 'cyto.cat', 'ITD', 'D835', 'Ras.Stat'] {}
Subset Entropy: 0.880636230815
Information gain from SEX: 0.000025
Information gain from PRIOR.MAL: 0.000009
Information gain from PRIOR.CHEMO: 0.000413
Information gain from PRIOR.XRT: 0.000898
Information gain from Infection: 0.000693
Information gain from cyto.cat: 0.098198
Information gain from ITD: 0.002720
Information gain from D835: 0.003004
Information gain from Ras.Stat: 0.006043
Branch: cyto.cat

Analyzing: resp.simple ['SEX', 'PRIOR.MAL', 'PRIOR.CHEMO', 'PRIOR.XRT', 'Infection', 'ITD', 'D835', 'Ras.Stat'] {'cyto.cat': u'diploid'}
Subset Entropy: 0.737489567214
Information gain from SEX: 0.009844
Information gain from PRIOR.MAL: 0.008397
Information gain from PRIOR.CHEMO: 0.019321
Information gain from PRIOR.XRT: 0.000014
Information gain from Infection: 0.000355
Information gain from ITD: 0.021205
Information gain from D835: 0.003763
Information gai

('cyto.cat',
 {-7: u'CR',
  -5: u'RESISTANT',
  8: u'CR',
  21: u'RESISTANT',
  u'-5,-7': u'CR',
  u'-5,-7,+8': u'RESISTANT',
  u'11q23': u'CR',
  u'IM': u'CR',
  u'Misc': u'CR',
  u'diploid': ('ITD',
   {u'NEG': ('Ras.Stat',
     {u'NEG': ('D835',
       {u'NEG': ('PRIOR.CHEMO', {u'NO': u'CR', u'YES': u'CR'}),
        u'POS': u'CR'}),
      u'NotDone': ('Infection', {u'No': u'CR', u'Yes': u'CR'}),
      u'POS': ('SEX',
       {u'F': ('PRIOR.XRT', {u'NO': u'CR', u'YES': u'CR'}),
        u'M': ('PRIOR.MAL', {u'NO': u'CR', u'YES': u'CR'})})}),
    u'POS': u'CR'}),
  u'inv9': u'CR',
  u't6;9': u'CR'})

'test-output/round-table.gv.pdf'

In [46]:
import numpy as np

def add_branch(Table,Dependent,Independent,Subset={},max_depth=-1,graph=Digraph(),min_subset=1):
    print '\nAnalyzing:',Subset#,Dependent,Independent,Subset
    #Select only the subset
    Table=Table.copy()
    for key in Subset.keys():
        if key in Categorical:
            Table=Table[Table[key]==Subset[key]]
        else:
            threeshold=float(Subset[key][1:])
            if Subset[key][0]=='<':
                Table=Table[Table[key]<threeshold]
            else:
                Table=Table[Table[key]>=threeshold]
                
    #If the subset has only one value, return the value
    if len(Table[Dependent].unique())==1:
        print 'Determinated Leaf:', Table[Dependent].unique()[0], len(Table[Dependent])
        return Table[Dependent].unique()[0]
    #If there are no more independent variables, return the most common value
    if len(Independent)==0 or max_depth==0 or len(Table)<min_subset:
        print 'Indeterminated Leaf:\n', Table[Dependent].value_counts()
        return Table[Dependent].value_counts().idxmax()
    
    #Calculate the entropy of the subset
    freq=[]
    for dval in Table[Dependent].unique():
        freq+=[sum(Table[Dependent]==dval)]
    Freq=[float(f)/sum(freq) for f in freq]
    E=0
    for f in Freq:
        E+=-f*np.log(f)/np.log(2)
    print 'Subset Entropy:', E
    Vars=[]
    
    #Calculate the entropy of each variable
    for ind in Independent:     
        if ind in Categorical:
            IG=E
            for ival in Table[ind].unique():
                SubTable=Table[Table[ind]==ival]
                #print SubTable
                freq=[]
                for dval in Table[Dependent].unique():
                    freq+=[sum(SubTable[Dependent]==dval)]
                Freq=[float(f)/sum(freq) for f in freq]
                #print Freq
                ES=0
                for f in Freq:
                    ES+=-f*np.log(f)/np.log(2) if f<>0 else 0
                #print ES
                IG-=float(len(SubTable))/len(Table)*ES
            #print 'Information gain from %s: %f'%(ind,IG)
            Vars+=[(IG,ind)]
        else:
            Threeshold=[]
            for ival in np.arange(min(Table[ind]),max(Table[ind]),(max(Table[ind])-min(Table[ind]))/100.0):
                IG=E
                SubTableA=Table[Table[ind]<ival]
                SubTableB=Table[Table[ind]>=ival]
                if len(SubTableA)<min_subset or len(SubTableB)<min_subset:
                    continue
                #print SubTable
                freq=[]
                for dval in Table[Dependent].unique():
                    freq+=[sum(SubTableA[Dependent]==dval)]
                Freq=[float(f)/sum(freq) for f in freq]
                #print Freq
                ES=0
                for f in Freq:
                    ES+=-f*np.log(f)/np.log(2) if f<>0 else 0
                #print ES
                IG-=float(len(SubTableA))/len(Table)*ES
                #print SubTable
                freq=[]
                for dval in Table[Dependent].unique():
                    freq+=[sum(SubTableB[Dependent]==dval)]
                Freq=[float(f)/sum(freq) for f in freq]
                #print Freq
                ES=0
                for f in Freq:
                    ES+=-f*np.log(f)/np.log(2) if f<>0 else 0
                #print ES
                IG-=float(len(SubTableB))/len(Table)*ES
                Threeshold+=[(IG,ival)]
            Threeshold.sort(reverse=True)
            #print Threeshold
            #break
            #print 'Information gain from %s: %f at theeshold:%f'%(ind,Threeshold[0][0],Threeshold[0][1])
            if len(Threeshold)>0:
                Vars+=[(Threeshold[0][0],ind,Threeshold[0][1])]
            else:
                Vars+=[(0,ind)]
    #Find best branch
    Vars.sort(reverse=True)
    #Use this branch only if there if some information gain:
    if Vars[0][0]>0:
        Selected_branch=Vars[0][1]
        if Selected_branch not in Categorical:
            Selected_threeshold=Vars[0][2]
        print 'Branch:', Selected_branch, Vars[0][0]
    else:
        print 'Indeterminated Leaf:\n', Table[Dependent].value_counts()
        return Table[Dependent].value_counts().idxmax()
    
    #Make a new branch
    Independent.remove(Selected_branch)
    choices={}
    Subset=Subset.copy()
    if Selected_branch in Categorical:
        for ival in Table[Selected_branch].unique():
                Subset.update({Selected_branch:ival})
                choices.update({ival:add_branch(Table,Dependent,Independent,Subset,max_depth-1)})
    else:
        Subset.update({Selected_branch:'<%f'%Selected_threeshold})
        choices.update({'<%f'%Selected_threeshold:add_branch(Table,Dependent,Independent,Subset,max_depth-1)})
        Subset.update({Selected_branch:'>%f'%Selected_threeshold})
        choices.update({'>%f'%Selected_threeshold:add_branch(Table,Dependent,Independent,Subset,max_depth-1)})
    return(Selected_branch,choices)

#Independent=list(Cars.keys())
#Independent.remove('Fast')
#Independent.remove('Model')
Independent=[v for v in All if v not in Dependent]
Table=Dream9_training
#,min_subset=10)

In [48]:
add_branch(Table,'resp.simple',Independent,max_depth=5)


Analyzing: {}
Subset Entropy: 0.880636230815
Branch: cyto.cat 0.0981982599493

Analyzing: {u'cyto.cat': u'diploid'}
Subset Entropy: 0.737489567214
Branch: CD13 0.116165357129

Analyzing: {u'cyto.cat': u'diploid', u'CD13': '<97.390000'}
Subset Entropy: 0.627819153483
Branch: TCF4 0.113388704117

Analyzing: {u'TCF4': '<-0.753153', u'cyto.cat': u'diploid', u'CD13': '<97.390000'}
Subset Entropy: 0.918295834054
Branch: PRKCA.pS657 0.918295834054

Analyzing: {u'TCF4': '<-0.753153', u'PRKCA.pS657': '<-0.575697', u'cyto.cat': u'diploid', u'CD13': '<97.390000'}
Determinated Leaf: CR 3

Analyzing: {u'TCF4': '<-0.753153', u'PRKCA.pS657': '>-0.575697', u'cyto.cat': u'diploid', u'CD13': '<97.390000'}
Determinated Leaf: RESISTANT 6

Analyzing: {u'TCF4': '>-0.753153', u'cyto.cat': u'diploid', u'CD13': '<97.390000'}
Subset Entropy: 0.468995593589
Branch: CD34 0.112113082169

Analyzing: {u'TCF4': '>-0.753153', u'CD34': '<27.720000', u'cyto.cat': u'diploid', u'CD13': '<97.390000'}
Determinated Leaf: CR

(u'cyto.cat',
 {-7: (u'TAZ', {'<0.565834': u'RESISTANT', '>0.565834': u'CR'}),
  -5: (u'ZNF346', {'<-0.157598': u'CR', '>-0.157598': u'RESISTANT'}),
  8: (u'MAP2K1_2.pS217_221',
   {'<0.896768': (u'ZNF296', {'<1.814725': u'CR', '>1.814725': u'RESISTANT'}),
    '>0.896768': u'RESISTANT'}),
  21: u'RESISTANT',
  u'-5,-7': (u'YWHAE', {'<-0.038393': u'RESISTANT', '>-0.038393': u'CR'}),
  u'-5,-7,+8': (u'YAP1p', {'<-1.244588': u'CR', '>-1.244588': u'RESISTANT'}),
  u'11q23': (u'YWHAZ', {'<-0.618852': u'RESISTANT', '>-0.618852': u'CR'}),
  u'IM': u'CR',
  u'Misc': (u'PRKAA1_2.pT172',
   {'<-0.419875': u'CR',
    '>-0.419875': (u'Age.at.Dx',
     {'<55.301400': u'CR',
      '>55.301400': (u'SMAD6',
       {'<0.288856': u'RESISTANT',
        '>0.288856': (u'TSC2',
         {'<0.942140': u'CR', '>0.942140': u'RESISTANT'})})})}),
  u'diploid': (u'CD13',
   {'<97.390000': (u'TCF4',
     {'<-0.753153': (u'PRKCA.pS657',
       {'<-0.575697': u'CR', '>-0.575697': u'RESISTANT'}),
      '>-0.753153': 

In [53]:
add_branch(Table[Table['Relapse'].notnull()],'Relapse',Independent,max_depth=5)


Analyzing: {}
Subset Entropy: 0.905044321288
Branch: LDH 0.0980360864372

Analyzing: {u'LDH': '<4236.300000'}
Subset Entropy: 0.851034066204
Branch: BCL2L1 0.0955226208217

Analyzing: {u'BCL2L1': '<-0.588506', u'LDH': '<4236.300000'}
Subset Entropy: 0.216396932451
Branch: SMAD2.pS245 0.216396932451

Analyzing: {u'BCL2L1': '<-0.588506', u'LDH': '<4236.300000', u'SMAD2.pS245': '<1.504456'}
Determinated Leaf: Yes 28

Analyzing: {u'BCL2L1': '<-0.588506', u'LDH': '<4236.300000', u'SMAD2.pS245': '>1.504456'}
Determinated Leaf: No 1

Analyzing: {u'BCL2L1': '>-0.588506', u'LDH': '<4236.300000'}
Subset Entropy: 0.943876757129
Branch: MAPT 0.11566555907

Analyzing: {u'BCL2L1': '>-0.588506', u'LDH': '<4236.300000', u'MAPT': '<-0.281456'}
Subset Entropy: 0.491237341824
Branch: CD7 0.274905030144

Analyzing: {u'BCL2L1': '>-0.588506', u'CD7': '<47.500000', u'LDH': '<4236.300000', u'MAPT': '<-0.281456'}
Subset Entropy: 0.242292189082
Branch: PDK1.pS241 0.242292189082

Analyzing: {u'BCL2L1': '>-0.588

(u'LDH',
 {'<4236.300000': (u'BCL2L1',
   {'<-0.588506': (u'SMAD2.pS245', {'<1.504456': u'Yes', '>1.504456': u'No'}),
    '>-0.588506': (u'MAPT',
     {'<-0.281456': (u'CD7',
       {'<47.500000': (u'PDK1.pS241',
         {'<-0.944619': u'No', '>-0.944619': u'Yes'}),
        '>47.500000': u'No'}),
      '>-0.281456': (u'PARP1',
       {'<-0.875881': u'No',
        '>-0.875881': (u'RPS6KB1',
         {'<0.618347': u'No', '>0.618347': u'Yes'})})})}),
  '>4236.300000': (u'TP53', {'<0.097830': u'No', '>0.097830': u'Yes'})})

In [54]:
add_branch(Table[Table['vital.status'].notnull()],'vital.status',Independent,max_depth=5)


Analyzing: {}
Subset Entropy: 0.728298978285
Branch: CD20 0.0698565894024

Analyzing: {u'CD20': '<19.800000'}
Subset Entropy: 0.691734419781
Branch: NR4A1 0.0593585753328

Analyzing: {u'CD20': '<19.800000', u'NR4A1': '<-0.491694'}
Subset Entropy: 0.985228136034
Branch: PIK3CA 0.272765700474

Analyzing: {u'PIK3CA': '<-0.038255', u'CD20': '<19.800000', u'NR4A1': '<-0.491694'}
Subset Entropy: 0.629249223856
Branch: SFN 0.458453829233

Analyzing: {u'SFN': '<-0.688107', u'PIK3CA': '<-0.038255', u'CD20': '<19.800000', u'NR4A1': '<-0.491694'}
Subset Entropy: 0.811278124459
Branch: XIAP 0.811278124459

Analyzing: {u'SFN': '<-0.688107', u'PIK3CA': '<-0.038255', u'XIAP': '<0.325151', u'CD20': '<19.800000', u'NR4A1': '<-0.491694'}
Determinated Leaf: A 3

Analyzing: {u'SFN': '<-0.688107', u'XIAP': '>0.325151', u'NR4A1': '<-0.491694', u'PIK3CA': '<-0.038255', u'CD20': '<19.800000'}
Determinated Leaf: D 1

Analyzing: {u'SFN': '>-0.688107', u'PIK3CA': '<-0.038255', u'CD20': '<19.800000', u'NR4A1': '

(u'CD20',
 {'<19.800000': (u'NR4A1',
   {'<-0.491694': (u'PIK3CA',
     {'<-0.038255': (u'SFN',
       {'<-0.688107': (u'XIAP', {'<0.325151': u'A', '>0.325151': u'D'}),
        '>-0.688107': u'D'}),
      '>-0.038255': (u'FIBRINOGEN',
       {'<538.490000': u'A',
        '>538.490000': (u'WTAP', {'<0.418420': u'D', '>0.418420': u'A'})})}),
    '>-0.491694': (u'BILIRUBIN',
     {'<0.161000': u'A',
      '>0.161000': (u'CD44',
       {'<-0.101966': u'D',
        '>-0.101966': (u'IGFBP2',
         {'<0.102110': u'D', '>0.102110': u'D'})})})}),
  '>19.800000': u'A'})

In [78]:
#Drawing trees
Tree=(u'CD20',
 {'<19.800000': (u'NR4A1',
   {'<-0.491694': (u'PIK3CA',
     {'<-0.038255': (u'SFN',
       {'<-0.688107': (u'XIAP', {'<0.325151': u'A', '>0.325151': u'D'}),
        '>-0.688107': u'D'}),
      '>-0.038255': (u'FIBRINOGEN',
       {'<538.490000': u'A',
        '>538.490000': (u'WTAP', {'<0.418420': u'D', '>0.418420': u'A'})})}),
    '>-0.491694': (u'BILIRUBIN',
     {'<0.161000': u'A',
      '>0.161000': (u'CD44',
       {'<-0.101966': u'D',
        '>-0.101966': (u'IGFBP2',
         {'<0.102110': u'D', '>0.102110': u'D'})})})}),
  '>19.800000': u'A'})

import random
def tree_plot(graph,tree,root='Start',decision='',level=0):
    if type(tree)==tuple:
        if level==0:
            graph.attr('node', shape='circle')
            graph.node(root)
        graph.attr('node', shape='diamond')
        graph.edge(root,tree[0],decision)
        for key in tree[1].keys():
            tree_plot(graph,tree=tree[1][key],root=tree[0],decision=key,level=level+1)
    else:
        new_name=''.join([random.choice('NEW_name') for _ in range(500)])
        graph.attr('node', shape='square')
        graph.node(new_name,label=tree)
        graph.edge(root,new_name,decision)


#Drawing the tree
from graphviz import Digraph

f = Digraph('Tree', filename='tree.gv')#, format='png')
f.body.extend(['rankdir=LR', 'size="8,5"'])
f.attr('node', shape='circle')

tree_plot(f,Tree)
f.view()

'tree.gv.pdf'